In [16]:
!pip install -e ../TrackSelectorDNN/ &> /tmp/install.log 

In [12]:
from cluster.RayJobManager import RayJobManager
from ray.job_submission import JobSubmissionClient
client = JobSubmissionClient(RAY_ADDRESS) 

In [19]:
RAY_ADDRESS  = "http://10.100.192.230:8265"
#RAY_ADDRESS  = "local"
ENTRYPOINT   = "python3 cluster/tune_grid.py"
WORKING_DIR  = "/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/"
PIP = {
    "packages": [
        "git+https://github.com/EmanueleCoradin/TrackSelectorDNN.git@main"
    ],
    "pip_check": False,
    "pip_install_options": ["--force-reinstall"]
}

ray_manager = RayJobManager(ray_address=RAY_ADDRESS)

In [20]:
job_id = ray_manager.submit_job(
    entrypoint=ENTRYPOINT,
    runtime_env={
        "working_dir": WORKING_DIR, 
        "pip": PIP, 
    }
    
)

2025-11-24 15:55:24,726	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_e3ba93a31747d730.zip.
2025-11-24 15:55:24,728	INFO packaging.py:588 -- Creating a file package for local module '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN-Ray/'.


Submitted Ray job: raysubmit_WHqw31LufBMTyABz


In [21]:
ray_manager.wait_until_done(job_id)

Job status: PENDING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
⚠️ Timeout waiting for job.


In [13]:
client.stop_job(job_id)

True

In [17]:
#Uncomment to run locally

from ray import tune
from TrackSelectorDNN.configs.schema import load_config
from TrackSelectorDNN.tune.trainable import trainable

base_cfg = load_config("base.yaml").model_dump()

search_space = {
    **base_cfg,  # include all validated defaults
    "lambda_sym": 0.1,
    "epochs": 5,
    "lr": tune.loguniform(1e-5, 1e-2),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "latent_dim": tune.choice([32, 64]),
    "netA_hidden_dim": tune.choice([32, 64]),
    "netB_hidden_dim": tune.choice([32, 64]),
    "pooling_type": tune.choice(["sum", "mean"]),
    "netA_hidden_layers": tune.choice([1, 2]),
    "netB_hidden_layers": tune.choice([1, 2]),
}

tuner = tune.Tuner(
    tune.with_resources(
        trainable,
        {"cpu": 1, "gpu": .25}  # adjust CPU/GPU numbers as needed
    ),
    param_space=search_space,
    tune_config=tune.TuneConfig(metric="val_loss", mode="min", num_samples=4),
)

results = tuner.fit()


2025-11-24 15:41:10,671	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/eos/user/e/ecoradin/ray_results/trainable_2025-11-24_15-38-42' in 0.0902s.
2025-11-24 15:41:10,685	INFO tune.py:1041 -- Total run time: 148.00 seconds (147.86 seconds for the tuning loop).


In [18]:
best_result = results.get_best_result(metric="val_loss", mode="min")

print("Best config found:")
print(best_result.config)

best_ckpt_dir = best_result.checkpoint
print("Best checkpoint path:", best_ckpt_dir)

Best config found:
{'hit_input_dim': 9, 'track_feat_dim': 25, 'latent_dim': 64, 'pooling_type': 'mean', 'netA_hidden_dim': 32, 'netA_hidden_layers': 1, 'netA_batchnorm': True, 'netA_activation': 'silu', 'netB_hidden_dim': 64, 'netB_hidden_layers': 1, 'netB_batchnorm': True, 'netB_activation': 'silu', 'lr': 0.00738941181419911, 'epochs': 5, 'batch_size': 128, 'patience': 15, 'delta': 0.0, 'base_checkpoint_directory': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/runs', 'dataset_type': 'production', 'dummy_load_path': None, 'train_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_train_dataset.pt', 'val_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_val_dataset.pt', 'test_path': '/eos/user/e/ecoradin/GitHub/TrackSelectorDNN/TrackSelectorDNN/data/tracks_test_dataset.pt', 'max_hits': 16, 'idxSymRecHitFeatures': [2, 7], 'idxSymRecoPixelTrackFeatures': [5, 7, 10, 22, 23], 'lambda_sym': 0.1, 'w_true': 0.6, 'w_fake': 2.91, 'wei